#### Import libraries

In [1]:
import numpy as np 
import pandas as pd
# import plotly.graph_objects as go
# import plotly.express as px
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import re
from xgboost import XGBClassifier,plot_importance
from sklearn.model_selection import cross_val_score,cross_val_predict,train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import imblearn
from imblearn.over_sampling import SMOTE
#Train 

import time

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)
pd.set_option("display.max_columns",None)

In [4]:
# file="../input"
file="files"

#### Read csv files to dataframe

In [5]:
df = pd.read_csv(file+"/train.csv")

In [6]:
train_y = df['m13']
df.drop(columns = 'm13',inplace = True)
df.columns

Index(['loan_id', 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12'],
      dtype='object')

In [36]:
df.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,response_time
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,2
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,2
2,742515242108,Z,Thornton-Davis,3.250,366000,180,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,601385667462,X,OTHER,4.750,135000,360,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,2
4,273870029961,X,OTHER,4.750,124000,360,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,2


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116058 entries, 0 to 116057
Data columns (total 28 columns):
loan_id                     116058 non-null int64
source                      116058 non-null object
financial_institution       116058 non-null object
interest_rate               116058 non-null float64
unpaid_principal_bal        116058 non-null int64
loan_term                   116058 non-null int64
origination_date            116058 non-null object
first_payment_date          116058 non-null object
loan_to_value               116058 non-null int64
number_of_borrowers         116058 non-null float64
debt_to_income_ratio        116058 non-null float64
borrower_credit_score       116058 non-null float64
loan_purpose                116058 non-null object
insurance_percent           116058 non-null float64
co-borrower_credit_score    116058 non-null float64
insurance_type              116058 non-null float64
m1                          116058 non-null int64
m2                  

#### Finding Response time from the date of loan to first payment

In [9]:
def response_time(df):
    diff = []
    for i in range(len(df['origination_date'])):
        temp = int(df['first_payment_date'][i].split('/')[0]) - int(df['origination_date'][i].split('-')[1])
        diff.append(temp)
    df.drop(columns = ['origination_date','first_payment_date'],inplace = True)
    df['response_time'] = diff
    return df

### Encoding using one hot encoding

In [10]:
def one_hot(df,columns,prefix):
#     columns_encoding = ['source','financial_institution','loan_purpose']
    columns_encoding = columns
    df2 = pd.get_dummies(df,columns = columns_encoding, prefix = ['source','fi','lp'])
    return df2

In [11]:
df2 = response_time(df)
df2 = one_hot(df,['source','financial_institution','loan_purpose'],['source','fi','lp'])
df2.columns

Index(['loan_id', 'interest_rate', 'unpaid_principal_bal', 'loan_term',
       'loan_to_value', 'number_of_borrowers', 'debt_to_income_ratio',
       'borrower_credit_score', 'insurance_percent',
       'co-borrower_credit_score', 'insurance_type', 'm1', 'm2', 'm3', 'm4',
       'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'response_time',
       'source_X', 'source_Y', 'source_Z', 'fi_Anderson-Taylor',
       'fi_Browning-Hart', 'fi_Chapman-Mcmahon', 'fi_Cole, Brooks and Vincent',
       'fi_Edwards-Hoffman', 'fi_Martinez, Duffy and Bird',
       'fi_Miller, Mcclure and Allen', 'fi_Nicholson Group', 'fi_OTHER',
       'fi_Richards-Walters', 'fi_Richardson Ltd',
       'fi_Romero, Woods and Johnson', 'fi_Sanchez, Hays and Wilkerson',
       'fi_Sanchez-Robinson', 'fi_Suarez Inc', 'fi_Swanson, Newton and Miller',
       'fi_Taylor, Hunt and Rodriguez', 'fi_Thornton-Davis',
       'fi_Turner, Baldwin and Rhodes', 'lp_A23', 'lp_B12', 'lp_C86'],
      dtype='object')

In [13]:
train_y.value_counts()

0    115422
1       636
Name: m13, dtype: int64

#### What we see is an imbalanced data. What we could try here is using SMOTE.

In [14]:
unique_ids = df2['loan_id']
df2.drop(columns = "loan_id",inplace = True)

In [15]:
df2.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12',
       'response_time', 'source_X', 'source_Y', 'source_Z',
       'fi_Anderson-Taylor', 'fi_Browning-Hart', 'fi_Chapman-Mcmahon',
       'fi_Cole, Brooks and Vincent', 'fi_Edwards-Hoffman',
       'fi_Martinez, Duffy and Bird', 'fi_Miller, Mcclure and Allen',
       'fi_Nicholson Group', 'fi_OTHER', 'fi_Richards-Walters',
       'fi_Richardson Ltd', 'fi_Romero, Woods and Johnson',
       'fi_Sanchez, Hays and Wilkerson', 'fi_Sanchez-Robinson',
       'fi_Suarez Inc', 'fi_Swanson, Newton and Miller',
       'fi_Taylor, Hunt and Rodriguez', 'fi_Thornton-Davis',
       'fi_Turner, Baldwin and Rhodes', 'lp_A23', 'lp_B12', 'lp_C86'],
      dtype='object')

In [16]:
retained = []
from scipy.stats import pearsonr
for i in df2.columns:
    corr, _ = pearsonr(df2[i],train_y)
    if abs(corr)>0.01:
        retained.append(i)
        print(i+'\t\t\t\t '+str(corr))

interest_rate				 0.054297748337329886
unpaid_principal_bal				 -0.017119713693303878
loan_term				 0.02350150259625623
loan_to_value				 0.016356698057617143
number_of_borrowers				 -0.03997893193327681
debt_to_income_ratio				 0.0380332430271289
borrower_credit_score				 -0.09421547991626723
co-borrower_credit_score				 -0.04265141994916702
m1				 0.0920536306133328
m2				 0.14692067142997856
m3				 0.15739400237425377
m4				 0.21895630047719536
m5				 0.2595289494162428
m6				 0.28771179017698006
m7				 0.3090718669636777
m8				 0.33936180205435057
m9				 0.3691377263202238
m10				 0.3678395993744353
m11				 0.4102133930470283
m12				 0.4664416294225055
source_X				 -0.010544394670021748
source_Y				 0.010279241930136466
lp_A23				 -0.02553164234786174
lp_B12				 0.015562882468223672
lp_C86				 0.01398352210011473


In [17]:
retained

['interest_rate',
 'unpaid_principal_bal',
 'loan_term',
 'loan_to_value',
 'number_of_borrowers',
 'debt_to_income_ratio',
 'borrower_credit_score',
 'co-borrower_credit_score',
 'm1',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'm10',
 'm11',
 'm12',
 'source_X',
 'source_Y',
 'lp_A23',
 'lp_B12',
 'lp_C86']

In [18]:
df3= df2[['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value','number_of_borrowers', 'debt_to_income_ratio','borrower_credit_score', 'co-borrower_credit_score',
 'm1',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'm10',
 'm11',
 'm12',
 'source_X',
 'source_Y',
 'lp_A23',
 'lp_B12',
 'lp_C86']]

In [19]:
df3.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'source_X', 'source_Y', 'lp_A23',
       'lp_B12', 'lp_C86'],
      dtype='object')

In [24]:
smt = SMOTE()
X_train, y_train = smt.fit_sample(df3, train_y)

In [25]:
X_train = pd.DataFrame(data = X_train, columns = df3.columns)

In [26]:
y_train = list(y_train)
np.bincount(y_train)

array([115422, 115422], dtype=int64)

#### Training with XGBoost classifier and evaluating with a train test split of 0.8

In [27]:
model = XGBClassifier(learning_rate = 0.055, max_depth = 4,n_jobs=-1)
train_x, test_x, train_Y, test_y = train_test_split(X_train,y_train,train_size=0.8)
model.fit(train_x,train_Y)
xgb_predict = model.predict(test_x)
    
print( "Train f1 score :: ", f1_score(train_Y, model.predict(train_x)))
print( "Test f1 score :: ", f1_score(test_y, xgb_predict))

Train f1 score ::  0.984770905019839
Test f1 score ::  0.9851864837621072


In [20]:
test_X = pd.read_csv("../input/test.csv")

In [21]:
months = {"Feb":2,"Mar":3,"Apr":4,"May":5}
diff2 = []
for i in range(len(test_X['origination_date'])):
    temp = months[test_X['first_payment_date'][i].split('-')[0]] - int(test_X['origination_date'][i].split('/')[1])
    diff2.append(temp)
test_X.drop(columns = ['origination_date','first_payment_date'],inplace = True)
test_X['response_time'] = diff2

In [22]:
test_X = one_hot(test_X,['source','financial_institution','loan_purpose'],['source','fi','lp'])

In [23]:
test_ids = test_X['loan_id']
test_X = test_X[['interest_rate',
 'unpaid_principal_bal',
 'loan_term',
 'loan_to_value',
 'number_of_borrowers',
 'debt_to_income_ratio',
 'borrower_credit_score',
 'co-borrower_credit_score',
 'm1',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'm10',
 'm11',
 'm12',
 'source_X',
 'source_Y',
 'lp_A23',
 'lp_B12',
 'lp_C86']]

In [24]:
test_X.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'source_X', 'source_Y', 'lp_A23',
       'lp_B12', 'lp_C86'],
      dtype='object')

**predict the delinquency for 13th month and save it in the submission file** 

In [25]:
xgb_predict_test = model.predict(test_X)

In [26]:
submission = pd.DataFrame(test_ids)
submission['m13'] = xgb_predict_test
submission.to_csv("sub.csv",index = False)

In [ ]:
# predictions = cross_val_predict(model, train_x,train_Y, cv=3)

# print(f1_score(train_Y,predictions))